In [2]:
import numpy as np
from scipy import ndimage

# Función que construye la grilla inicial

def create_grid(d, grid_size, m, pattern='random'):
    """
    Crea una grilla n-dimensional
    
    Parámetros:
    - d: número de dimensiones
    - grid_size: tamaño de cada dimensión (int o lista de int)
    - m: valor máximo de estado (estados van de 0 a m + 1)
    - pattern: tipo de patrón inicial ('random', 'central', etc.)
    """
    # Convertir grid_size a una tupla si es un solo número
    if isinstance(grid_size, int):
        grid_size = tuple([grid_size] * d)
    else:
        grid_size = tuple(grid_size)
    
    # Crear la grilla según el patrón solicitado
    if pattern == 'random':
        # Grilla con valores aleatorios entre 0 y m
        return array_lib.random.randint(0, m+1, grid_size)
    
    elif pattern == 'central':
        # Grilla con valor alto en el centro y ceros alrededor
        grid = array_lib.zeros(grid_size, dtype=int)
        center = tuple(s // 2 for s in grid_size)
        
        # Colocar valor m en el centro
        indices = tuple(slice(c-1, c+2) for c in center)
        grid[indices] = m
        return grid
    
    elif pattern == 'checkerboard':
        # Patrón de tablero de ajedrez
        indices = array_lib.indices(grid_size)
        sum_indices = array_lib.sum(indices, axis=0)
        return (sum_indices % 2) * m
    
    else:
        # Patrón predeterminado: valores 0
        return array_lib.zeros(grid_size, dtype=int)
    


def update_grid(grid, m, radius=1):
    """
    Actualiza la grilla según las reglas del autómata celular generalizado.
    
    Parámetros:
    - grid: La grilla n-dimensional actual
    - m: Valor máximo de estado (los estados van de 0 a m)
    - radius: Radio de la vecindad
    
    Retorna:
    - La grilla actualizada
    """
    # Calcular suma de vecinos usando convolución
    dims = grid.ndim
    kernel_size = 2*radius + 1
    
    # Crear kernel: todos 1s excepto el centro
    kernel = np.ones([kernel_size] * dims, dtype=int)
    center = tuple(radius for _ in range(dims))
    kernel[center] = 0
    
    # Aplicar convolución con bordes toroidales
    neighbor_sum = ndimage.convolve(grid, kernel, mode='wrap')
    
    # Calcular el valor máximo posible de la suma de vecinos
    # Número total de vecinos = (2*r+1)^d - 1
    total_neighbors = (2*radius + 1)**dims - 1
    max_sum = m * total_neighbors
    
    # Crear los 3 intervalos equiespaciados
    interval_size = max_sum / 3
    
    # Crear máscaras para cada intervalo
    interval1 = neighbor_sum < interval_size
    interval2 = (neighbor_sum >= interval_size) & (neighbor_sum < 2*interval_size)
    interval3 = neighbor_sum >= 2*interval_size
    
    # Crear una copia de la grilla para los resultados
    new_grid = grid.copy()
    
    # Aplicar las reglas:
    # - Si está en el primer intervalo, disminuye en 1
    # - Si está en el segundo intervalo, aumenta en 1
    # - Si está en el tercer intervalo, disminuye en 1
    
    # Disminuir en 1 para el primer intervalo (sin ir por debajo de 0)
    new_grid[interval1] = np.maximum(0, grid[interval1] - 1)
    
    # Aumentar en 1 para el segundo intervalo (sin superar m)
    new_grid[interval2] = np.minimum(m, grid[interval2] + 1)
    
    # Disminuir en 1 para el tercer intervalo (sin ir por debajo de 0)
    new_grid[interval3] = np.maximum(0, grid[interval3] - 1)
    
    return new_grid

def simulate_automaton(d, grid_size, m, radius=1, iterations=100, pattern='random'):
    """
    Ejecuta la simulación del autómata celular generalizado.
    
    Parámetros:
    - d: Número de dimensiones
    - grid_size: Tamaño de la grilla en cada dimensión
    - m: Número máximo de estados (0 a m)
    - radius: Radio de vecindad
    - iterations: Número de iteraciones
    - pattern: Patrón inicial
    
    Retorna:
    - Una lista con la grilla en cada iteración
    """
    # Crear la grilla inicial
    grid = create_grid(d, grid_size, m, pattern)
    
    # Lista para almacenar la evolución
    evolution = [grid.copy()]
    
    # Único bucle for permitido: iteraciones temporales
    for i in range(iterations):
        grid = update_grid(grid, m, radius)
        evolution.append(grid.copy())
    
    return evolution

In [3]:
# Prueba con una grilla 2D simple
test_grid = np.array([[1, 2], [3, 4]])
print("Grilla original:")
print(test_grid)

# Calcular la suma de vecinos
dims = test_grid.ndim
radius = 1
kernel_size = 2*radius + 1
kernel = np.ones([kernel_size] * dims, dtype=int)
center = tuple(radius for _ in range(dims))
kernel[center] = 0
neighbor_sum = ndimage.convolve(test_grid, kernel, mode='wrap')

print("\nSuma de vecinos:")
print(neighbor_sum)

Grilla original:
[[1 2]
 [3 4]]

Suma de vecinos:
[[26 22]
 [18 14]]
